In [8]:
# !sudo apt-get install python3.7
# !sudo apt-get update -y
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
# !sudo update-alternatives --config python3

In [9]:
!pip install numpy
!pip install pyopenms
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.9 MB/s eta 0:00:00a 0:00:01


In [97]:
import re

EXPERIMENT_NAME = "OVFF368"
# reading file
file1 = open(f'./content/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}.txt', 'r')
oligo_list = [line for line in file1.readlines() if re.search("^[a-zA-Z][1-9]", line)]
import csv

header = ['where\tcode\tseq\tDMT\tcomment\tlength\t']
data = [row.strip() for row in oligo_list]


with open(f'./content/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_oligos.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    writer = csv.writer(f, quotechar='\"', delimiter='\n', quoting=csv.QUOTE_NONE, escapechar='\\')
    # write the data
    writer.writerow(data)

In [98]:
import pandas as pd

df = pd.read_csv(f'./content/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_oligos.csv', delimiter='\t')
# drop NaN seq
df = df[df['seq'].notna()]
oligo_list = df['seq'].to_list()

In [99]:
from Bio.Seq import Seq
from Bio.SeqUtils import molecular_weight

# Bio.SeqUtils.molecular_weight(seq, seq_type='DNA', double_stranded=False, circular=False, monoisotopic=False)

from pyopenms import *

# # reading file
# file1 = open('./OVFF317_oligos_seq.txt', 'r')
# oligo_list = file1.readlines()

mass_list=[]
# https://biopython.org/docs/1.75/api/Bio.SeqUtils.html
for oligo_seq in oligo_list: 
  try:
    my_dna = Seq(oligo_seq)
  except:
    print(oligo_seq)
  try:
    print(f"{oligo_seq.strip()}\t" + "%0.2f" % (molecular_weight(my_dna)-79.97))
    mass_list+=["%0.2f" % (molecular_weight(my_dna)-79.97)]
  except ValueError:
    print(f"{oligo_seq.strip()}\t" + "'N' is not a valid unambiguous letter for DNA")
    mass_list+=["NaN"]
    
df['monoisotopic_mass']=mass_list
export_dataframe=df[['where', 'code', 'length', 'seq', 'monoisotopic_mass']]
export_dataframe.to_csv(f'./content/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_calculated.csv', sep = '\t', index=False)

CGT CCG GCG TAG AGG ATC GAG ATC TTA ATA CGA CTC GGT ATA GGG GAA TTG	15859.18
TTC TAG AGG GGA ATT GTT ATC CGC TCA CAA TTC CCC TAT ACC GAG TCG TAT TAA	16545.64
TGA GCG GAT AAC AAT TCC CCT CTA GAA ATA ATT TTG TTT AAC TTT AAG AAG GAG	16665.77
TTG GGC GGA CGG AAG CAT GGT ATA TCT CCT TCT TAA AGT TAA ACA AAA TTA T	16064.37
ATA TAC CAT GCT TCC GTC CGC CCA AGC GCC CTC CCT CCT CAA TCC CA	14096.05
CCA GTG CCG CGA AGT CGT CGG TGG GAT TGA GGA GGG AGG GCG C	13471.61
CCG ACG ACT TCG CGG CAC TGG GCA ATA TTG CCT GGC TGT GGA TGA ACT	14800.48
GCC AGT CGC GGT GCA TGG GAG AGT TCA TCC ACA GCC AGG CAA TAT TGC	14818.51
CTC CCA TGC ACC GCG ACT GGC CGG TGC ATC TGC TCG CAC GC	12459.97
GTT GAA TCG GCG CGA GCG TGT TGC GTG CGA GCA GAT GCA CCG	13052.35
AAC ACG CTC GCG CCG ATT CAA CTG GGC CAA TAC ATT CTG CTG CGA TG	14384.24
GCA GTA TGC AAC CGG CAC GTC ATT GCA TCG CAG CAG AAT GTA TTG GCC CA	15379.88
CAA TGA CGT GCC GGT TGC ATA CTG CAG CTG GGC CCT AAT GGA CGC CG	14481.28
CGC CAT GAC ATA GGA GAG TTC GGT GTC GGC GTC CAT 

In [100]:
# export_dataframe


In [101]:
# molecular_weight(Seq("TG"))

In [102]:
# print([ord(char) - 96 for char in input('Write Text: ').lower()])

In [103]:
# seq = "CCCGGGGTACCACGGAGAGATGGTGGAAATCATCTTTCTCGTGGGCATCCTTGATGGCCACCTCGTCGGAAGTGCCCACGAGGATGACAGCAATGCCAATGCTGGGGGGGCTCTTCTGAGAACGAGCTCTGCTGCCTGACACGGCCAGGACGGCCAACACCAACCAGAACTTGGGAGAACAGCACTCCGCTCTGGGCTTCATCTTCAACTCGTCGACTCCCTGCAAACACAAAGAAAGAGCATGTTAAAATAGGATCTACATCACGTAACCTGTCTTAGAAGAGGCTAGATACTGCAATTCAAGGACCTTATCTCCTTTCATTGAGCACCAAACCCAACTCCATCTACCAGCCTACTCTCTTATCTCTGGTATT"


In [104]:
# seq.upper()

In [105]:
# len(seq)
# my_dna = Seq(seq)
# molecular_weight(my_dna)

In [106]:
EXPERIMENT_NAME = "N4-1A"
# reading file
import pandas as pd
df_calculated =  pd.read_csv(f'./content/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_calculated.csv', delimiter='\t', decimal=".")

df = pd.read_csv(f'./content/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_concentrations.csv', delimiter='\t', decimal=".")

"""
strip the text everywhere on data frame
"""
cols = df.select_dtypes(object).columns
df[cols] = df[cols].apply(lambda x: x.str.strip())
"""
and again
"""
cols = df_calculated.select_dtypes(object).columns
df_calculated[cols] = df_calculated[cols].apply(lambda x: x.str.strip())

"""
now merge cleaned dataframes
"""
merge = pd.merge(df, df_calculated, on='where', how='inner', suffixes=('', '_drop'))
#Drop the duplicate columns
merge.drop([col for col in merge.columns if 'drop' in col], axis=1, inplace=True)

df = merge

df.to_csv(f'./content/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_merged_input.csv', encoding='UTF8')

In [90]:
import sys

# from Bio.Alphabet import IUPAC
from Bio.Seq import Seq

# seq01 = Seq("acgt", IUPAC.unambiguous_dna)
# # arguments for script (input file name, PCR volume, required number of PCR reactions, SPE elution
# # volume)
# scheme_file_name = sys.argv[1]
# reaction_volume = float(sys.argv[2])
# number_of_reactions = float(sys.argv[3])
# elution_volume = float(sys.argv[4])


# calculation of oligo extinction coefficients
def e260_func(sequence):
    Coefficients = {
        "a": 15.4,
        "c": 7.4,
        "g": 11.5,
        "t": 8.7,
        "aa": 13.7,
        "ac": 10.6,
        "ag": 12.5,
        "at": 11.4,
        "ca": 10.6,
        "cc": 7.3,
        "cg": 9,
        "ct": 7.6,
        "ga": 12.6,
        "gc": 8.8,
        "gg": 10.8,
        "gt": 10,
        "ta": 11.7,
        "tc": 8.1,
        "tg": 9.5,
        "tt": 8.4,
    }
    
    Coefficients =  {key.upper(): val for key, val in Coefficients.items()}
    e260 = 0
    sequence = Seq(sequence.replace(" ", ""))
    
    for i in range(0, len(sequence) - 1):
        e260 = e260 + 2 * Coefficients[sequence[i] + sequence[i + 1]]
    for i in range(0, len(sequence) - 2):
        e260 = e260 - Coefficients[sequence[i + 1]]
    return e260

Yield = 99

# e260 = e260_func(sec)

for seq in df["seq"]:
    e260 = e260_func(seq)
    A260 = df[df["seq"] == seq]["A260"]
    monoisotopic_mass = df[df["seq"] == seq]["monoisotopic_mass"]
    
    corrected_concentration = A260 / (e260 + (e260 / 2) * (1 - (Yield / 100)) / (Yield / 100)) * 1000000
    print(corrected_concentration*monoisotopic_mass/1000) # (mM–1cm–1)
# # Input format for TBIO calculation:
# # Name Sequence Yield, % Target Concentration, nMA260 (1 cm)
# oligos_table = [
#     [
#         "Name",
#         "Sequence",
#         "Yield, %",
#         "e260, mM-1cm-1",
#         "A260, (1cm)",
#         "Concentration, nM",
#         "Target Concentration, nM",
#         "V, ul",
#         "Vnorm, ul",
#     ]
# ]
# volumes = []
# total_volume_for_reaction = 0
# total_volume_for_mixing = 0
# A260_afterSPE_exp = 0
# # reading input file
# with open(scheme_file_name) as scheme_file:
#     lines = scheme_file.readlines()
# lines = lines[1 : len(lines)]
# for line in lines:
#     Name = line.split("\t")[0]
#     Sequence = line.split("\t")[1]
#     Yield = float(100)
#     e260 = e260_func(line.split("\t")[1].lower())
#     A260 = float(line.split("\t")[4])
#     Concentration = (
#         A260 / (e260 + (e260 / 2) * (1 - (Yield / 100)) / (Yield / 100)) * 1000000
#     )
#     Target_Concentration = float(line.split("\t")[3])
#     V = reaction_volume / (Concentration / Target_Concentration)
#     Vnorm = reaction_volume / (Concentration / Target_Concentration)
#     oligos_table.append(
#         [
#             Name,  # 0
#             Sequence,
#             Yield,
#             str(round(e260, 5)),
#             str(A260),
#             str(round(Concentration, 5)),
#             str(Target_Concentration),
#             str(round(V, 5)),
#             str(round(Vnorm, 5)),
#         ]

0    369980.256036
dtype: float64
1    407593.595234
dtype: float64
2    343650.840143
dtype: float64
3    316037.367192
dtype: float64
4    385647.593818
dtype: float64
5    242878.01283
dtype: float64
6    405244.086218
dtype: float64
7    400325.077794
dtype: float64
8    450957.322159
dtype: float64
9    304171.868479
dtype: float64
10    405154.805934
dtype: float64
11    367654.821594
dtype: float64
12    325213.657378
dtype: float64
13    403630.488123
dtype: float64
14    267246.484616
dtype: float64
15    383337.370657
dtype: float64
16    307785.278617
dtype: float64
17    379231.119001
dtype: float64
18    361800.772731
dtype: float64
19    384971.01346
dtype: float64
20    344888.374815
dtype: float64
21    339281.193537
dtype: float64
22    377084.673694
dtype: float64
23    374681.806058
dtype: float64
24    381163.958006
dtype: float64
25    326787.755729
dtype: float64
26    367910.285093
dtype: float64
27    431094.213234
dtype: float64
28    277465.301907
dtype: float

In [69]:
seq  = Seq("TGG TCG CTA TAT CCC GTA CAG ACA AGG CTA TTC CAG CTT TAC GTG AAC TAT TCA TTA GTG CT".replace(" ", ""))

In [91]:
e260 = e260_func(seq)
A260 = df[df["seq"] == seq]["A260"]
monoisotopic_mass = df[df["seq"] == seq]["monoisotopic_mass"]

corrected_concentration = A260 / (e260 + (e260 / 2) * (1 - (Yield / 100)) / (Yield / 100)) * 1000000
print(corrected_concentration*monoisotopic_mass/1000) # (mM–1cm–1)

45    250895.815855
dtype: float64


In [92]:
e260  # (M–1cm–1)

338.8000000000002

In [93]:
A260 # безразмерно

45    8.381
Name: A260, dtype: float64

In [94]:
monoisotopic_mass

45    10193.63
Name: monoisotopic_mass, dtype: float64

In [95]:
# factor  = 33 ng/mkl

In [96]:
corrected_concentration = A260 / e260 
corrected_concentration*monoisotopic_mass

45    252.162966
dtype: float64

In [77]:
df[df["seq"] == seq]

,where,Date,Sample Name,Nucleic Acid(ng/uL),A260/A280,A260/A230,A260,A280,Nucleic Acid Factor,Baseline Correction (nm),Baseline Absorbance,code,length,seq,monoisotopic_mass


In [89]:
corrected_concentration = A260 / e260